In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Data Preprocessing and Loading

In [13]:
from src.data.data import DataProcessor, TimeSeriesDataset, TimeSeriesDataLoader

# Preprocess data
data_processor = DataProcessor(start='2021-03-30 00:00:00', 
                               end='2021-03-31 00:00:00', 
                               verbose=True)
data_processor.prepare_data(filename='bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv', 
                            exclude_columns_detrend=['SMA', 'RSI', 'MACD', 'log_ret', 'percent_ret'],
                            exclude_columns_scale=['log_ret'])

# # Create dataloaders
train_loader, val_loader, test_loader = data_processor.create_dataloaders(window=10, 
                                                                          batch_size=64,
                                                                          exclude_input_columns=[''])

data_processor.databundle.train.head()

Data loaded --------- data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv
Using data from 2021-03-30 00:00:00 to 2021-03-31 00:00:00 with 1440 rows and 12 columns


,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,SMA,RSI,MACD,log_ret,percent_ret
Timestamp,,,,,,,,,,,,
2021-03-30 00:00:00,0.774739,0.793285,0.791302,0.808348,0.077207,0.077625,0.785713,0.254462,0.520251,0.374126,0.001223,1.512183
2021-03-30 00:01:00,0.746803,0.735067,0.710674,0.685440,0.030152,0.031030,0.730351,0.253190,0.235106,0.320573,-0.003268,-4.122627
2021-03-30 00:02:00,0.672719,0.751875,0.700307,0.765878,0.071704,0.072152,0.766804,0.253666,0.419462,0.316518,0.002201,2.743318
2021-03-30 00:03:00,0.761304,0.765591,0.771190,0.764775,0.023572,0.024529,0.753513,0.252559,0.420015,0.313214,0.000007,-0.015307
2021-03-30 00:04:00,0.749900,0.759519,0.748191,0.774439,0.023224,0.024181,0.730557,0.252938,0.443036,0.315564,0.000297,0.348134
